In [10]:
# Creating spark session
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import os

# Creating a spark session
spark=(
    SparkSession.builder
    .master('local[2]')
    .appName('transformApp')
    .config('spark.jars','postgresql-42.7.4.jar')
    .config('spark.driver.extraClassPath','postgresql-42.7.4.jar')
    .getOrCreate()
)

data_quality={
    'null_count':{
        "null_count_df_movie": int,
        "null_count_df_distributor": int,
        "null_count_df_genre": int,
        "null_count_df_filmmaker": int,
        "null_count_df_boxoffice": int
    }
}


# Loading the data into spark DataFrames
df_boxoffice=spark.read.json('run_20241126/collection_parsed_20241126.json',multiLine=True)
df_movie_raw=spark.read.json('run_20241126/movies_parsed_20241126.json',multiLine=True)
df_filmmaker_raw=spark.read.json('run_20241126/filmmaker_parsed_20241126.json',multiLine=True)
df_genre_raw=spark.read.json('run_20241126/genre_parsed_20241126.json',multiLine=True)
df_distributor_raw=spark.read.json('run_20241126/distributor_parsed_20241126.json',multiLine=True)

# df_boxoffice=spark.read.json('run_20241106/collection_parsed_2024116.json',multiLine=True)
# df_movie_raw=spark.read.json('run_20241106/movies_parsed_2024116.json',multiLine=True)
# df_filmmaker_raw=spark.read.json('run_20241106/filmmaker_parsed_2024116.json',multiLine=True)
# df_genre_raw=spark.read.json('run_20241106/genre_parsed_2024116.json',multiLine=True)
# df_distributor_raw=spark.read.json('run_20241106/distributor_parsed_2024116.json',multiLine=True)



# Shaping the data in desired format from raw DataFrames
df_movie=(
    df_movie_raw.select(explode(df_movie_raw.content).alias('movies'),'extracted_at','run_date')
        .select('movies.*','extracted_at','run_date')       
)

df_genre=(
    df_genre_raw.select(explode(df_genre_raw.content).alias('genre'),'extracted_at','run_date')
        .select('genre.*','extracted_at','run_date')      
)

df_filmmaker=(
    df_filmmaker_raw.select(explode(df_filmmaker_raw.content).alias('filmmaker'),'extracted_at','run_date')
        .select('filmmaker.*','extracted_at','run_date')       
)

df_distributor=(
    df_distributor_raw.select(explode(df_distributor_raw.content).alias('distributor'),'extracted_at','run_date')
        .select('distributor.*','extracted_at','run_date')       
)





# Bridge table b/w movies and genre dimension
bridge_movie_genre=df_genre.select('genre_id','movie_id','run_date')

# Bridge table b/w movies and filmmaker dimension
bridge_filmmaker_movie=(
    df_movie.withColumn('filmmaker',explode(df_movie['filmmakers']))
        .select('filmmaker','movie_id','run_date')
        .select('filmmaker.filmmaker_id','filmmaker.role','movie_id','run_date')     
)





df_genre=(
    df_genre.dropDuplicates(['genre_id']) # deduplication based on "genre_id"
            .drop('movie_id') # drop the inappropriate field "movie_id"
)
df_filmmaker=(
    df_filmmaker.dropDuplicates(['filmmaker_id']) # deduplication based on "filmmaker_id"
                .drop('movie_id') # drop the inappropriate field "movie_id"
)
df_distributor=(
    df_distributor.dropDuplicates(['distributor_id']) # deduplication based on "distributor_id"
)

# Handling NULL values in the dimension tables ------------------------------------------------------------------------------------------------------------------------------

data_quality['null_count']['null_count_df_movie']=df_movie.where('movie_id is null OR release_id is null').count()
data_quality['null_count']['null_count_df_distributor']=df_distributor.where('distributor_id is null').count()
data_quality['null_count']['null_count_df_genre']=df_genre.where('genre_id is null').count()
data_quality['null_count']['null_count_df_filmmaker']=df_filmmaker.where('filmmaker_id is null').count()
data_quality['null_count']['null_count_df_boxoffice']=df_boxoffice.where('release_id is null').count()
data_quality['null_count']['null_count_bridge_movie_genre']=bridge_movie_genre.where('genre_id is null OR movie_id is null').count()
data_quality['null_count']['null_count_bridge_filmmaker_movie']=bridge_filmmaker_movie.where('filmmaker_id is null OR movie_id is null').count()

df_movie=df_movie.dropna(subset='movie_id')
df_boxoffice=df_boxoffice.dropna(subset='release_id')
df_genre=df_genre.dropna(subset='genre_id')
df_filmmaker=df_filmmaker.dropna(subset='filmmaker_id')
df_distributor=df_distributor.dropna(subset='distributor_id')
bridge_movie_genre = bridge_movie_genre.dropna(subset=['genre_id','movie_id'],how='any')
bridge_filmmaker_movie = bridge_filmmaker_movie.dropna(subset=['filmmaker_id','movie_id'],how='any')

# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Adding the movie_id field in the df_boxoffice DataFrame using the df_movie DataFrame
df_boxoffice=df_boxoffice.join(other=df_movie.select(['release_id','movie_id']),on='release_id',how='left')


# Cleaning fields to remove "$" and "," charachters from the fields
from pyspark.sql.functions import regexp_replace
df_boxoffice=(
    df_boxoffice.withColumns({
        "collection_domestic":regexp_replace('collection_domestic',r'[\$\,]',''),
        "days_post_release":regexp_replace('days_post_release',r'[\,]',''),
        "num_theatres":regexp_replace('num_theatres',r'[\,]','')
    })
)

df_distributor=(
    df_distributor.withColumn("total_movies",regexp_replace("total_movies",r'\,',''))
)


# Adding calculated field "post_release_days" to the df_movie DataFrame
from pyspark.sql.functions import to_date,date_diff,current_date
df_movie=df_movie.withColumn('post_release_days',
                    date_diff(
                        start=to_date('release_date_id',
                        format="yyyyMMdd"),
                        end=current_date()
                    )
)

# Casting fields of all DataFrames to appropriate data types ----------------------------------------------------------------------------------------------------------------
df_movie=(
    df_movie.withColumns({
    "release_date_id":df_movie.release_date_id.cast(IntegerType()),
    "duration":df_movie.duration.cast(IntegerType()),
    "widest_release":df_movie.widest_release.cast(IntegerType()),
    "imdb_rating":df_movie.imdb_rating.cast(FloatType()),
    "num_of_rating":df_movie.num_of_rating.cast(IntegerType()),
    "extracted_at":df_movie.extracted_at.cast(IntegerType()),
    "run_date":df_movie.run_date.cast(IntegerType())
    })
)

df_genre=(
    df_genre.withColumns({
        "total_movies":df_genre.total_movies.cast(IntegerType()),
        "extracted_at":df_genre.extracted_at.cast(IntegerType()),
        "run_date":df_genre.run_date.cast(IntegerType())
    })
)

df_filmmaker=(
    df_filmmaker.withColumns({
        "dob":df_filmmaker.dob.cast(DateType()),
        "total_movies":df_filmmaker.total_movies.cast(IntegerType()),
        "extracted_at":df_filmmaker.extracted_at.cast(IntegerType()),
        "run_date":df_filmmaker.run_date.cast(IntegerType())
    })
)

df_distributor=(
    df_distributor.withColumns({
        "total_movies":df_distributor.total_movies.cast(IntegerType()),
        "extracted_at":df_distributor.extracted_at.cast(IntegerType()),
        "run_date":df_distributor.run_date.cast(IntegerType())
    })
)

df_boxoffice=(
    df_boxoffice.withColumns({
        "run_date":df_boxoffice.run_date.cast(IntegerType()),
        "collection_domestic":df_boxoffice.collection_domestic.cast(FloatType()),
        "days_post_release":df_boxoffice.days_post_release.cast(IntegerType()),
        "num_of_releases":df_boxoffice.num_of_releases.cast(IntegerType()),
        "rank":df_boxoffice.rank.cast(IntegerType()),
        "num_theatres":df_boxoffice.num_theatres.cast(IntegerType()),
        "extracted_at":df_boxoffice.extracted_at.cast(IntegerType())
    })
)

bridge_filmmaker_movie = bridge_filmmaker_movie.withColumn('run_date',col('run_date').cast(IntegerType()))

bridge_movie_genre = bridge_movie_genre.withColumn('run_date',col('run_date').cast(IntegerType()))

# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Adding the field "age" to DataFrame "df_filmmaker"
df_filmmaker=df_filmmaker.withColumn('age',(date_diff(end=current_date(),start='dob')/365).cast(IntegerType()))

# Dropping the "filmmakers" and "genres" fields as the bridge table are created.
df_movie=df_movie.drop('filmmakers','genres')

# Similar to the above step, dropping the movie_id field from df_distributor
df_distributor=df_distributor.drop('movie_id')

df_boxoffice=df_boxoffice.drop('movie_name')


# Renaming the field run_data to effective_from from all the table which includes it ----------------------------------------------------------------------------------------------------------------
df_boxoffice=df_boxoffice.withColumnRenamed(existing='run_date',new='effective_from')
df_movie=df_movie.withColumnRenamed(existing='run_date',new='effective_from')
df_genre=df_genre.withColumnsRenamed(
    {
        'run_date':'effective_from',
        'top_movie_NumOfRating':'top_movie_numofrating',
        'top_movie_US_BoxOffice':'top_movie_us_boxoffice'
    }
)
df_filmmaker=df_filmmaker.withColumnsRenamed(
    {
        'run_date':'effective_from',
        'top_movie_NumOfRating':'top_movie_numofrating',
        'top_movie_US_BoxOffice':'top_movie_us_boxoffice'
    }
)
df_distributor=df_distributor.withColumnsRenamed(
    {
        'run_date':'effective_from',
        'top_movie_NumOfRating':'top_movie_numofrating',
        'top_movie_US_BoxOffice':'top_movie_us_boxoffice'
    }
)

bridge_filmmaker_movie = bridge_filmmaker_movie.withColumnRenamed(existing='run_date',new='effective_from')

bridge_movie_genre = bridge_movie_genre.withColumnRenamed(existing='run_date',new='effective_from')
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


In [5]:
print(f"Total rows in df_movie: {df_movie.count()}")
print(f"Total rows in df_genre: {df_genre.count()}")
print(f"Total rows in df_filmmaker: {df_filmmaker.count()}")
print(f"Total rows in df_distributor: {df_distributor.count()}")
print(f"Total rows in bridge_filmmaker_movie: {bridge_filmmaker_movie.count()}")
print(f"Total rows in bridge_movie_genre: {bridge_movie_genre.count()}")
print(f"Total rows in df_boxoffice: {df_boxoffice.count()}")

Total rows in df_movie: 24
Total rows in df_genre: 64
Total rows in df_filmmaker: 129
Total rows in df_distributor: 16
Total rows in bridge_filmmaker_movie: 151
Total rows in bridge_movie_genre: 140
Total rows in df_boxoffice: 24


In [11]:
# Loading the data to postgres
url='jdbc:postgresql://localhost:5432/movie_data?user=jayesh'


df_movie.write.jdbc(url=url,table='dim_movies_staging',mode='append')
# df_genre.write.jdbc(url=url,table='dim_genre_staging',mode='append')
# df_filmmaker.write.jdbc(url=url,table='dim_filmmakers_staging',mode='append')
# df_distributor.write.jdbc(url=url,table='dim_distributor_staging',mode='append')
# bridge_filmmaker_movie.write.jdbc(url=url,table='bridge_movie_filmmaker_staging',mode='append')
# bridge_movie_genre.write.jdbc(url=url,table='bridge_movie_genre_staging',mode='append')
# df_boxoffice.write.jdbc(url=url,table='fact_domestic_collection',mode='append')

In [7]:
df_genre.show()

+---------+------------------+---------------------+----------------------+------------+------------+--------------+
| genre_id|        genre_name|top_movie_numofrating|top_movie_us_boxoffice|total_movies|extracted_at|effective_from|
+---------+------------------+---------------------+----------------------+------------+------------+--------------+
|in0000001|            Action|            tt0468569|             tt2488496|       58032|    20241128|      20241126|
|in0000002|       Action Epic|            tt0468569|             tt2488496|         194|    20241128|      20241126|
|in0000008|         Superhero|            tt0468569|             tt4154796|        1164|    20241128|      20241126|
|in0000009|    Sword & Sandal|            tt0172495|             tt0325980|          64|    20241128|      20241126|
|in0000012|         Adventure|            tt1375666|             tt2488496|       29957|    20241128|      20241126|
|in0000015|    Adventure Epic|            tt1375666|            

In [ ]:
#TODO
    # 1. Add fields listed in the data model to the corresponding DataFrames
    # 2. Check for possible undesired values in every DataFrames' fields and fix it by replacing it with the desired value so everything can be casted properly.
